In [1]:
import os

In [2]:
%pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC'

In [5]:
import dagshub
dagshub.init(repo_owner='AryanDhanuka10', repo_name='End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as AryanDhanuka10

Initialized MLflow to track repo "AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC"

Repository AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC initialized!

🏃 View run rumbling-ape-214 at: https://dagshub.com/AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC.mlflow/#/experiments/0/runs/df8e55b0d956472eb54639f823ba2ecf
🧪 View experiment at: https://dagshub.com/AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC.mlflow/#/experiments/0


In [6]:
import tensorflow as tf

2025-03-28 18:05:00.843165: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 18:05:00.867620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-28 18:05:00.867648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-28 18:05:00.868346: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 18:05:00.872864: I tensorflow/core/platform/cpu_feature_guar

In [7]:
import os

model_path = "/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC/artifacts/training/model.h5"
if os.path.exists(model_path):
    print("✅ Model file exists!")
else:
    print("❌ Model file NOT found! Retrain and save the model.")


✅ Model file exists!


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluated_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC/artifacts/training/model.h5",
            training_data="/home/aryan-dhanuka/Backup/Python Data Science/AI Projects/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC/artifacts/data_ingestion/Chest-CT-Scan-data/train",
            mlflow_url="https://dagshub.com/AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC.mlflow/#/experiments/0?searchFilter=&orderByKey=attributes.start_time&orderByAsc=false&startTime=ALL&lifecycleFilter=Active&modelVersionFilter=All+Runs&datasetsFilter=W10%3D",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [ ]:


@dataclass
class EvaluationConfig:
    path_of_model: Path
    training_data: str
    params_image_size: tuple
    params_batch_size: int
    mlflow_url: str
    all_params: dict

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30  # Keep validation split
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"  # Ensure proper multi-class classification
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        print("Valid Class Indices:", self.valid_generator.class_indices)
        print("Detected classes:", self.valid_generator.class_indices)
        print("Number of detected classes:", len(self.valid_generator.class_indices))
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(path)
        model.summary()  # Ensure model supports 4 classes
        return model

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        # Ensure 4 output classes
        num_classes = len(self.valid_generator.class_indices)
        if num_classes != 4:
            raise ValueError(f"Expected 4 classes, but found {num_classes}. Check dataset!")

        self.score = self.model.evaluate(self.valid_generator)
        print(f"Loss: {self.score[0]}, Accuracy: {self.score[1]}")
        self.save_score()

    import json

    def save_json(path: Path, data: dict):
        with open(path, "w") as f:
            json.dump(data, f, indent=4)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model")
            else:
                mlflow.keras.log_model(self.model, "model")
    



#### **Pipeline**

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluated_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-03-28 18:05:01,902: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-28 18:05:01,903: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-28 18:05:01,904: INFO: common: created directory at: artifacts]


2025-03-28 18:05:01.937670: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-28 18:05:01.957302: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-28 18:05:01.957466: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2025-03-28 18:05:02.624930: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


12/12 [==============================] - 5s 172ms/step - loss: 0.8951 - accuracy: 0.6154
Loss: 0.8951007127761841, Accuracy: 0.6153846383094788
[2025-03-28 18:05:07,731: INFO: common: json file saved at: scores.json]


2025/03/28 18:05:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmp1gb0n3n6/model/data/model/assets
[2025-03-28 18:05:09,907: INFO: builder_impl: Assets written to: /tmp/tmp1gb0n3n6/model/data/model/assets]


2025/03/28 18:05:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run victorious-wren-852 at: https://dagshub.com/AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC.mlflow/#/experiments/0/runs/c0c7bf92616f4607937a55c21a11fcf1
🧪 View experiment at: https://dagshub.com/AryanDhanuka10/End-To-End-ML_Project-Chest-Cancer-Detection-Using-MLOps-and-DVC.mlflow/#/experiments/0
